In [ ]:
!apt update && apt install -y libsm6 libxext6 libfontconfig1 libxrender1
!pip3 install --upgrade pip
!pip3 install opencv-python tqdm albumentations scikit-learn scikit-image segmentation_models_pytorch

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import numpy as np
import time
import cv2
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.backends.cudnn as cudnn
import pandas as pd
import torch.optim as optim
import random
import sys
import glob
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp
import sys
sys.path.insert(0, 'over9000/')

os.system(f"""git clone https://github.com/mgrankin/over9000.git""")
from ralamb import Ralamb
from radam import RAdam
from ranger import Ranger
from lookahead import LookaheadAdam
from over9000 import Over9000
from tqdm.notebook import tqdm
from tqdm import tqdm_notebook as tqdm

#For Transformations
import cv2
import glob
import tifffile as tiff
from torch.utils.data import Dataset, DataLoader, sampler
import albumentations as aug
from albumentations import (HorizontalFlip, VerticalFlip, ShiftScaleRotate, Normalize, Resize, Compose,Cutout, GaussNoise, RandomRotate90, Transpose, RandomBrightnessContrast, RandomCrop)
from albumentations.pytorch import ToTensor

# For Meter
from sklearn.metrics import fbeta_score, precision_recall_fscore_support, accuracy_score
from sklearn.metrics import jaccard_similarity_score as jaccard_score
from sklearn.model_selection import train_test_split

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
seed_everything(43)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Replace with whatever extension your mask has, as in our case it is png

In [ ]:
class SensoVisionDataset(Dataset):
    def __init__(self, phase, path=os.getcwd(), img_ext=".jpg", mask_ext=".png"):
        self.transforms = get_transforms(phase)
        self.phase = phase
        self.path = path
        self.img_ext = img_ext
        self.mask_ext = mask_ext
        train = os.listdir(self.path+'/Images/')  # >>>>>>>>>>>>>>>> Replace here
        self.train, self.val = train_test_split(train, test_size = 0.2)
        self.dim = (256, 256)
    def __getitem__(self, idx):
        if self.phase == 'train':
            img = cv2.imread(self.path+'/Images/'+self.train[idx])
            img = cv2.resize(img, self.dim, interpolation = cv2.INTER_NEAREST)  # Resizing to fit with EfficientNet
            mask = cv2.imread(self.path+'/Masks/' + self.train[idx].split('.')[0]+self.mask_ext, 0)
            mask = cv2.resize(mask, self.dim, interpolation = cv2.INTER_NEAREST)
        elif self.phase == 'val':
            img = cv2.imread(self.path+'/Images/'+self.val[idx])
            img = cv2.resize(img, self.dim, interpolation = cv2.INTER_NEAREST)  # Resizing to fit with EfficientNet
            mask = cv2.imread(self.path+'/Masks/' + self.val[idx].split('.')[0]+self.mask_ext, 0)
            mask = cv2.resize(mask, self.dim, interpolation = cv2.INTER_NEAREST)
        mask = (mask != 0)*255 
        mask = mask.astype(np.uint8)
        augmented = self.transforms(image=img, mask=mask)
        img = augmented['image']
        mask = augmented['mask']
        return img, mask

    def __len__(self):
        if self.phase == 'train':
            return len(self.train)
        else:
            return len(self.val)

In [ ]:
def get_transforms(phase):
    list_transforms = []
    if phase == "train":
        list_transforms.extend([
             aug.Flip()
            ])

    list_transforms.extend(
        [
#             Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225], p=1),
            ToTensor(),
        ]
    )
    list_trfms = Compose(list_transforms)
    return list_trfms

In [ ]:
def provider(phase, path=os.getcwd(), img_ext=".jpg", mask_ext=".png", batch_size=8, num_workers=0):
    '''Returns dataloader for the model training'''
    image_dataset = SensoVisionDataset(phase, path, img_ext, mask_ext)
        
    dataloader = DataLoader(
        image_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=False,
        shuffle=True,   
    )

    return dataloader

In [ ]:
dl = provider('train', "./storage/Dataset/", ".png", ".png")

In [ ]:
for img, mask in dl:
    print(img.shape)
    print(mask.shape)
    break

In [ ]:
for i in range(img.shape[0]):
    image = img[i].permute(1, 2, 0).cpu().numpy()
    msk = mask[i].squeeze(0).cpu().numpy()
    plt.imshow(image)
    plt.show()
    plt.imshow(msk)
    plt.show()

In [ ]:
class BCEDiceLoss(nn.Module):
    __name__ = 'bce_dice_loss'

    def __init__(self, eps=1e-7, beta=2., activation='sigmoid', ignore_channels=None, threshold=None):
        super().__init__()
        self.bce = nn.BCEWithLogitsLoss(reduction='mean')
        self.beta = beta
        self.eps = eps
        self.threshold = threshold
        self.ignore_channels = ignore_channels
        self.activation = smp.utils.base.Activation(activation)

    def forward(self, y_pr, y_gt):
        bce = self.bce(y_pr, y_gt)
        y_pr = self.activation(y_pr)
        dice = 1 - smp.utils.functional.f_score(
            y_pr, y_gt,
            beta=self.beta,
            eps=self.eps,
            threshold=self.threshold,
            ignore_channels=self.ignore_channels,
        )
        return dice + bce

class DiceLoss(nn.Module):
    __name__ = 'dice_loss'

    def __init__(self, eps=1e-7, beta=2., activation='sigmoid', ignore_channels=None, threshold=None):
        super().__init__()
        self.beta = beta
        self.eps = eps
        self.threshold = threshold
        self.ignore_channels = ignore_channels
        self.activation = smp.utils.base.Activation(activation)

    def forward(self, y_pr, y_gt):
        y_pr = self.activation(y_pr)
        dice = 1 - smp.utils.functional.f_score(
            y_pr, y_gt,
            beta=self.beta,
            eps=self.eps,
            threshold=self.threshold,
            ignore_channels=self.ignore_channels,
        )
        return dice


class Tversky(nn.Module):
    
    def __init__(self, alpha = 0.5, beta = 0.5, smooth = 1e-10):
        super(Tversky, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.smooth = 1e-10

    def forward(self, y_pred, y_true):
        y_pred = y_pred.type(torch.FloatTensor)
        y_true = y_true.type(torch.FloatTensor)
        y_pred = F.sigmoid(y_pred)
        num_classes = y_true.size(1)
        bs = y_true.size(0)
        tversky = 0
        for i in range(num_classes):
            y_t_i = y_true[:,i,...]
            y_p_i = y_pred[:,i,...]
            y_t_i = y_t_i.reshape(bs,-1)
            y_p_i = y_p_i.reshape(bs,-1)
            truepos = torch.sum(y_t_i*y_p_i, dim=1)
            fp_fn = self.alpha*torch.sum(y_p_i * (1 - y_t_i), dim=1) + self.beta * torch.sum((1 - y_p_i) * y_t_i, dim=1)
            tversky_i = (truepos + self.smooth) / ((truepos + self.smooth) + fp_fn)
            tversky+=torch.mean(tversky_i)
        return 1-tversky/num_classes

class JaccardLoss(nn.Module):
    __name__ = 'bce_dice_loss'

    def __init__(self, eps=1e-7, beta=1., activation='sigmoid', ignore_channels=None, threshold=None):
        super().__init__()
        self.beta = beta
        self.eps = eps
        self.threshold = threshold
        self.ignore_channels = ignore_channels
        self.activation = smp.utils.base.Activation(activation)

    def forward(self, y_pr, y_gt):
        y_pr = self.activation(y_pr)
        jaccard = 1 - smp.utils.functional.jaccard(
            y_pr, y_gt,
            eps=self.eps,
            threshold=self.threshold,
            ignore_channels=self.ignore_channels,
        )
        return jaccard

class BCEDiceJaccardLoss(nn.Module):
    __name__ = 'bce_dice_loss'

    def __init__(self, eps=1e-7, beta=1., activation='sigmoid', ignore_channels=None, threshold=None):
        super().__init__()
        self.bce = nn.BCEWithLogitsLoss(reduction='mean')
        self.beta = beta
        self.eps = eps
        self.threshold = threshold
        self.ignore_channels = ignore_channels
        self.activation = smp.utils.base.Activation(activation)

    def forward(self, y_pr, y_gt):
        bce = self.bce(y_pr, y_gt)
        y_pr = self.activation(y_pr)
        dice = 1 - smp.utils.functional.f_score(
            y_pr, y_gt,
            beta=self.beta,
            eps=self.eps,
            threshold=self.threshold,
            ignore_channels=self.ignore_channels,
        )
        jaccard = 1 - smp.utils.functional.jaccard(
            y_pr, y_gt,
            eps=self.eps,
            threshold=self.threshold,
            ignore_channels=self.ignore_channels,
        )
        return (dice + bce + jaccard)/3

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
def get_dice(y_true, y_pred):
    dice = 0
    batch_size = y_true.shape[0]
    channel_size = y_true.shape[1]
    for i in range(batch_size):
      for j in range(channel_size):
        y_tr = y_true[i][j]
        y_pr = y_pred[i][j]
        # print(y_tr.shape, y_pr.shape)
        y_tr = y_tr.reshape(-1,)
        y_pr = y_pr.reshape(-1,)
        # print(y_tr.shape, y_pr.shape)
        dice += f1_score(y_tr, y_pr)/(batch_size*channel_size)
    return dice

def get_f2(y_true, y_pred):
    f2 = 0
    batch_size = y_true.shape[0]
    channel_size = y_true.shape[1]
    for i in range(batch_size):
      for j in range(channel_size):
        y_tr = y_true[i][j]
        y_pr = y_pred[i][j]
        y_tr = y_tr.reshape(-1,)
        y_pr = y_pr.reshape(-1,)
        f2 += fbeta_score(y_tr, y_pr, beta=0.5)/(batch_size*channel_size)
    return f2

##>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> CHANGE THRESHOLD FOR IOU HERE
def soft_jaccard_score(y_pred: torch.Tensor, y_true: torch.Tensor, smooth=0.0, eps=1e-7, threshold=0.6) -> torch.Tensor:

    assert y_pred.size() == y_true.size()
    bs = y_true.size(0)
    num_classes = y_pred.size(1)
    dims = (0, 2)
    y_pred = (y_pred>threshold).float()
    y_true = y_true.view(bs, num_classes, -1)
    y_pred = y_pred.view(bs, num_classes, -1)
    
    if dims is not None:
        intersection = torch.sum(y_pred * y_true, dim=dims)
        cardinality = torch.sum(y_pred + y_true, dim=dims)
    else:
        intersection = torch.sum(y_pred * y_true)
        cardinality = torch.sum(y_pred + y_true)

    union = cardinality - intersection
    jaccard_score = (intersection + smooth) / (union.clamp_min(eps) + smooth)
    return jaccard_score.mean().item()

class Meter:
    '''A meter to keep track of iou and dice scores throughout an epoch'''
    def __init__(self, phase, epoch):
        self.threshold = 0.5 # <<<<<<<<<<< here's the threshold
        self.base_dice_scores = []
        self.iou_scores = []
        self.f2_scores = []
        self.phase = phase


    def update(self, y_true, y_preds):
        # applying sigmoid and getting labels as 0 and 1
        # print(y_preds)
        probs_sig = torch.sigmoid(y_preds)
        # print(probs_sig)
        # print(y_preds)
        iou = soft_jaccard_score(probs_sig, y_true)
        
        y_true = y_true.float().detach().cpu().numpy()
        preds = y_preds.float().detach().cpu().numpy()
        preds = (preds > self.threshold).astype('uint8')
        dice = get_dice(y_true, preds)
        f2 = get_f2(y_true, preds)

        self.base_dice_scores.append(dice)
        self.f2_scores.append(f2)
        self.iou_scores.append(iou)

    def get_metrics(self):
        dice = np.mean(self.base_dice_scores)
        f2 = np.mean(self.f2_scores)
        iou = np.nanmean(self.iou_scores)
        return dice, iou, f2

def epoch_log(phase, epoch, epoch_loss, meter, start):
    '''logging the metrics at the end of an epoch'''
    dice, iou, f2 = meter.get_metrics()
    print("Loss: %0.4f | IoU: %0.4f | dice: %0.4f | f2_score: %0.4f" % (epoch_loss, iou, dice, f2))
    return dice, iou, f2

In [ ]:
class Trainer(object):
    '''This class takes care of training and validation of our model'''
    def __init__(self,model, path, img_ext, mask_ext, optim, loss, lr, bs, name, shape=512, crop_type=0):
        self.num_workers = 4
        self.batch_size = {"train": bs, "val": 1}
        self.accumulation_steps = bs // self.batch_size['train']
        self.lr = lr
        self.path = path
        self.img_ext = img_ext
        self.mask_ext = mask_ext
        self.loss = loss
        self.optim = optim
        self.num_epochs = 0
        self.best_val_loss = 1
        self.best_val_dice = 0
        self.best_val_iou = 0
        self.phases = ["train", "val"]
        self.device = torch.device("cuda:0")
        torch.set_default_tensor_type("torch.cuda.FloatTensor")
        self.net = model
        self.name = name
        self.do_cutmix = True
        self.loss_classification = torch.nn.CrossEntropyLoss()
        if self.loss == 'BCE':
            self.criterion = torch.nn.BCEWithLogitsLoss()
        elif self.loss == 'BCE+DICE':
            self.criterion = BCEDiceLoss(threshold=None)  #MODIFIED
        elif self.loss == 'IOU':
            self.criterion = JaccardLoss(threshold=None)  #MODIFIED
        elif self.loss == 'TVERSKY':
            self.criterion = Tversky()
        elif self.loss == 'Dice' or self.loss == 'DICE':
            self.criterion = DiceLoss()
        elif self.loss == 'BCE+DICE+IOU':
            self.criterion = BCEDiceJaccardLoss(threshold=None)
        else:
            raise(Exception(f'{self.loss} is not recognized. Please provide a valid loss function.'))

        # Optimizers
        if self.optim == 'Over9000':
            self.optimizer = Over9000(self.net.parameters(),lr=self.lr)
        elif self.optim == 'Adam':
            self.optimizer = torch.optim.Adam(self.net.parameters(),lr=self.lr)
        elif self.optim == 'RAdam':
            self.optimizer = Radam(self.net.parameters(),lr=self.lr)
        elif self.optim == 'Ralamb':
            self.optimizer = Ralamb(self.net.parameters(),lr=self.lr)
        elif self.optim == 'Ranger':
            self.optimizer = Ranger(self.net.parameters(),lr=self.lr)
        elif self.optim == 'LookaheadAdam':
            self.optimizer = LookaheadAdam(self.net.parameters(),lr=self.lr)
        else:
            raise(Exception(f'{self.optim} is not recognized. Please provide a valid optimizer function.'))
            
        self.scheduler = ReduceLROnPlateau(self.optimizer, factor=0.5, mode="min", patience=4, verbose=True, min_lr = 1e-5)
        self.net = self.net.to(self.device)
        cudnn.benchmark = True
        
        self.dataloaders = {
            phase: provider(
                phase=phase,
                path = self.path,
                img_ext = self.img_ext,
                mask_ext = self.mask_ext,
                num_workers=0,
            )
            for phase in self.phases
        }
        self.losses = {phase: [] for phase in self.phases}
        self.dice_scores = {phase: [] for phase in self.phases}
        self.iou_scores = {phase: [] for phase in self.phases}
        self.f2_scores = {phase: [] for phase in self.phases}

    def freeze(self):
        for  name, param in self.net.encoder.named_parameters():
            if name.find('bn') != -1:
                param.requires_grad=True
            else:
                param.requires_grad=False
                
    def seed_everything(self, seed):
        random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True

    def load_model(self, name, path='models/'):
        state = torch.load(path+name, map_location=lambda storage, loc: storage)
        self.net.load_state_dict(state['state_dict'])
        self.optimizer.load_state_dict(state['optimizer'])
        print("Loaded model with dice: ", state['best_dice'])
            
    def unfreeze(self):
        for param in self.net.parameters():
            param.requires_grad=True
     
    def forward(self, images, targets):
        images = images.to(self.device)
        targets = targets.to(self.device)
        preds = self.net(images)
        # print(preds)
        loss = self.criterion(preds, targets)
        return loss, targets, preds

    def iterate(self, epoch, phase):
        meter = Meter(phase, epoch)
        start = time.strftime("%H:%M:%S")
        print(f"Starting epoch: {epoch} | phase: {phase} | ⏰: {start}")
        batch_size = self.batch_size[phase]
        dataloader = self.dataloaders[phase]
        running_loss = 0.0
        total_batches = len(dataloader)
        tk0 = tqdm(dataloader, total=total_batches)
        self.optimizer.zero_grad()
        for itr, batch in enumerate(tk0):
            if phase == "train" and self.do_cutmix:
                images, targets = self.cutmix(batch, 0.5)
            else:
                images, targets = batch
            seg_loss, outputs, preds = self.forward(images, targets)
            loss = seg_loss / self.accumulation_steps
            if phase == "train":
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.net.parameters(), 1)
                if (itr + 1 ) % self.accumulation_steps == 0:
                    self.optimizer.step()
                    self.optimizer.zero_grad()
            running_loss += loss.item()
            meter.update(outputs, preds)
            tk0.set_postfix(loss=(running_loss / ((itr + 1))))
        epoch_loss = (running_loss * self.accumulation_steps) / total_batches
        dice, iou, f2 = epoch_log(phase, epoch, epoch_loss, meter, start)
        self.losses[phase].append(epoch_loss)
        self.dice_scores[phase].append(dice)
        self.iou_scores[phase].append(iou)
        self.f2_scores[phase].append(f2)
        torch.cuda.empty_cache()
        return epoch_loss, dice, iou

    def train_end(self):
        train_dice = self.dice_scores["train"]
        train_iou = self.iou_scores["train"]
        train_f2 = self.f2_scores["train"]
        train_loss = self.losses["train"]
        
        val_dice = self.dice_scores["val"]
        val_iou = self.iou_scores["val"]
        val_f2 = self.f2_scores["val"]
        val_loss = self.losses["val"]

        df_data=np.array([train_loss, train_dice, train_iou, train_f2, val_loss, val_dice, val_iou, val_f2]).T
        df = pd.DataFrame(df_data,columns = ["train_loss", "train_dice", "train_iou", "train_f2", "val_loss", "val_dice", "val_iou", "val_f2"])
        df.to_csv('logs/'+self.name+'.csv')

    def fit(self, epochs):
        self.num_epochs+=epochs
        for epoch in range(self.num_epochs-epochs, self.num_epochs):
            self.net.train()
            self.iterate(epoch, "train")
            state = {
                "epoch": epoch,
                "best_loss": self.best_val_loss,
                "best_dice": self.best_val_dice,
                "best_iou": self.best_val_iou,
                "state_dict": self.net.state_dict(),
                "optimizer": self.optimizer.state_dict(),
            }
            self.net.eval()
            with torch.no_grad():
                val_loss, val_dice, val_iou = self.iterate(epoch, "val")
                self.scheduler.step(val_loss)
            if val_loss < self.best_val_loss:
                print("* New optimal found according to Validation loss!, saving state *")
                state["best_loss"] = self.best_val_loss = val_loss
                os.makedirs('models/', exist_ok=True)
                torch.save(state, 'models/'+self.name+'_best_loss.pth')
            if val_dice > self.best_val_dice:
                print("* New optimal found according to Dice Score!, saving state *")
                state["best_dice"] = self.best_val_dice = val_dice
                os.makedirs('models/', exist_ok=True)
                torch.save(state, 'models/'+self.name+'_best_dice.pth')
            if val_iou > self.best_val_iou:
                print("* New optimal found according to IOU Score!, saving state *")
                state["best_iou"] = self.best_val_iou = val_iou
                os.makedirs('models/', exist_ok=True)
                torch.save(state, 'models/'+self.name+'_best_iou.pth')
            print()
            self.train_end()

In [ ]:
def TRAIN(MODEL, ENCODER, OPTIMIZER, LOSS, path, img_ext, mask_ext):
    if(MODEL == 'Unet'):
        model = smp.Unet(ENCODER, encoder_weights='imagenet', classes=1, activation=None)
    elif(MODEL == 'FPN'):
        model = smp.FPN(ENCODER, encoder_weights='imagenet', classes=1, activation=None)
    elif(MODEL == 'Linknet'):
        model = smp.Linknet(ENCODER, encoder_weights='imagenet', classes=1, activation=None)
    
    model_trainer = Trainer(model = model, path=path, img_ext=img_ext, mask_ext=mask_ext, optim = OPTIMIZER, loss = LOSS, lr = 1e-3, bs = 8, name = ENCODER+'_'+MODEL+'_'+LOSS+'_'+OPTIMIZER)
    model_trainer.do_cutmix = False
    model_trainer.freeze()
    model_trainer.fit(10)
    model_trainer.do_cutmix = False
    model_trainer.unfreeze()
    model_trainer.fit(10)
    model_trainer.do_cutmix = False
    model_trainer.freeze()
    model_trainer.fit(5)
    return model

## EDIT HERE

In [ ]:
Model = ['FPN'] # 'Unet', 'Linknet'
Encoder = ['efficientnet-b4'] #'efficientnet-b3', 'efficientnet-b4', 'efficientnet-b5' 
Optimizer = ['Ranger']
Loss = ['BCE+DICE+IOU'] # 'Dice', 'IOU', 
path = "./storage/Dataset/" # Should contain 'Images' folder and 'Makss' folder
img_ext = '.png' # Check and edit
mask_ext = '.png' #Check and edit


for model in Model:
    for encoder in Encoder:
        for optimizer in Optimizer:
            for loss in Loss:
                m = TRAIN(model, encoder, optimizer, loss, path, img_ext, mask_ext)

In [ ]:
# Best dice: 8520 IOU: 7534, f2: 8339

### Inference